In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os, sys, platform

print(f"python version: {sys.version}")
print(f"os type: {os.name}")
print(f"os: {platform.system()}")
print(f"os release: {platform.release()}")

from os.path import join, dirname, basename

train_count = 0
test_count  = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if "dcm" in filename:
            if 'test_images' in dirname:
                test_count += 1
            else:
                train_count += 1
        else:
            print(os.path.join(dirname, filename))

print("total train:", train_count)
print("total test :", test_count)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
os type: posix
os: Linux
os release: 5.15.65+
/kaggle/input/rsna-breast-cancer-detection/sample_submission.csv
/kaggle/input/rsna-breast-cancer-detection/train.csv
/kaggle/input/rsna-breast-cancer-detection/test.csv
/kaggle/input/monai-whl/monai-1.1.0-202212191849-py3-none-any.whl
/kaggle/input/torchio/torchio-0.18.86-py2.py3-none-any.whl
/kaggle/input/torchio/Deprecated-1.2.13-py2.py3-none-any.whl
/kaggle/input/torchio/shellingham-1.5.0.post1-py2.py3-none-any.whl
total train: 54706
total test : 4


In [2]:
! pip install --no-index --find-links=/kaggle/input/monai-whl monai
! pip install --no-index --find-links=/kaggle/input/torchio torchio

Looking in links: /kaggle/input/monai-whl
Processing /kaggle/input/monai-whl/monai-1.1.0-202212191849-py3-none-any.whl
Looking in links: /kaggle/input/torchio
Processing /kaggle/input/torchio/torchio-0.18.86-py2.py3-none-any.whl
Processing /kaggle/input/torchio/Deprecated-1.2.13-py2.py3-none-any.whl
Processing /kaggle/input/torchio/shellingham-1.5.0.post1-py2.py3-none-any.whl


In [3]:
# !pip install monai torchio
import pydicom
import SimpleITK as sitk
import cv2
import torch
import tensorflow
import matplotlib.pyplot as plt
# import monai
import monai
# import torchio
import torchio as tio

print(monai.__version__)
print(tio.__version__)

SRC = "/kaggle/input/rsna-breast-cancer-detection"

1.1.0
0.18.86


In [4]:
print(pydicom.__version__)
!pwd
print('------')
!ls /kaggle/*

2.3.0
/kaggle/working
------
/kaggle/input:
monai-whl  rsna-breast-cancer-detection  torchio

/kaggle/lib:
kaggle

/kaggle/src:
script.ipynb

/kaggle/working:
__notebook__.ipynb


In [5]:
dcm = pydicom.read_file("/kaggle/input/rsna-breast-cancer-detection/test_images/10008/68070693.dcm")

dcm.pixel_array.shape

(2776, 2082)

In [6]:
sample = pd.read_csv(join(SRC,"sample_submission.csv"))
sample

,prediction_id,cancer
0,10008_L,0.021168
1,10008_R,0.021168


In [7]:
test_exl = pd.read_csv(join(SRC,"test.csv"))
test_exl

,site_id,patient_id,image_id,laterality,view,age,implant,machine_id,prediction_id
0,2,10008,736471439,L,MLO,81,0,21,10008_L
1,2,10008,1591370361,L,CC,81,0,21,10008_L
2,2,10008,68070693,R,MLO,81,0,21,10008_R
3,2,10008,361203119,R,CC,81,0,21,10008_R


In [8]:
results = []
prediction_id = np.unique(test_exl.prediction_id.values)
for idx in prediction_id:
    results.append([idx, np.random.randint(0,1000)/1000])
    
results = pd.DataFrame(results, columns=['prediction_id','cancer'])
results

,prediction_id,cancer
0,10008_L,0.877
1,10008_R,0.385


In [9]:
results.to_csv('./sample_submission.csv', index=False)
results.to_csv('./submission.csv', index=False)